In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import copy
from scipy import stats
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
import random
from sklearn import datasets
from sklearn.semi_supervised import LabelPropagation
from sklearn.semi_supervised import LabelSpreading
from sklearn.metrics import accuracy_score

%matplotlib inline

In [2]:
def draw(model, X, y):

    h = .02

    # Plot the decision boundary. For that, we will assign a color to each
    # point in the mesh [x_min, x_max]x[y_min, y_max].
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])

    Z = Z.reshape(xx.shape)
    plt.figure(1, figsize=(4, 3))
    plt.pcolormesh(xx, yy, Z, cmap=plt.cm.Paired)

    plt.scatter(X[:, 0], X[:, 1], c=y, edgecolors='k', cmap=plt.cm.Paired)
    plt.xlabel('Sepal length')
    plt.ylabel('Sepal width')

ZADANIE: Wypróbuj działanie poznanych klasyfikatorów na iris. Weź zbiór 10 etykietowanych przykładów. Użyj domyślnych parametrów dla różnych kerneli

In [3]:


rng = np.random.RandomState(42)
iris = datasets.load_iris()
labels = np.copy(iris.target)

random.seed(1)
num_labels = 10
num_unlabeled = len(iris.target)-num_labels
inds_unlabeled = random.sample(range(len(iris.target)), num_unlabeled)
labels[inds_unlabeled] = -1

len(labels[labels > -1])

10

Zobaczmy domyśle działanie na iris

In [4]:


label_prop_model = LabelPropagation()
label_prop_model.fit(iris.data, labels)
y_pred = label_prop_model.predict(iris.data)
print(accuracy_score(y_pred, iris.target))


0.9533333333333334


In [5]:
label_prop_model = LabelPropagation(kernel='knn')
label_prop_model.fit(iris.data, labels)
y_pred = label_prop_model.predict(iris.data)
print(accuracy_score(y_pred, iris.target))

0.3333333333333333


C:\Users\marek\Anaconda3\lib\site-packages\sklearn\semi_supervised\label_propagation.py:277: RuntimeWarning: invalid value encountered in true_divide
  self.label_distributions_ /= normalizer
C:\Users\marek\Anaconda3\lib\site-packages\sklearn\semi_supervised\label_propagation.py:288: ConvergenceWarning: max_iter=1000 was reached without convergence.
  category=ConvergenceWarning


Jeśli tylko mamy dość danych etykietowanych to warto uczyć hiperparametrów. Nistety, sklearn nie daje automatycznych metod uczenia hipeparametrów metod semi-supervised. Musimy to implementować sami.

ZADANIE: Napisz metodę która robi CV dla LabelSpreading po ilościach sąsiadów

In [7]:
from sklearn import datasets
from sklearn.semi_supervised import LabelPropagation
from sklearn.semi_supervised import LabelSpreading

rng = np.random.RandomState(42)
iris = datasets.load_iris()
labels = np.copy(iris.target)

random.seed(1)
num_labels = 50
num_unlabeled = len(iris.target)-num_labels
inds_unlabeled = random.sample(range(len(iris.target)), num_unlabeled)
labels[inds_unlabeled] = -1

len(labels[labels > -1])

X= iris.data
y=labels
print(X.shape)
print(y)

(150, 4)
[-1 -1 -1 -1 -1 -1  0 -1  0  0  0 -1 -1 -1  0 -1 -1  0  0 -1 -1  0 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1  0  0 -1  0 -1 -1 -1 -1  0  0 -1  0 -1 -1 -1 -1
 -1  0 -1 -1 -1 -1 -1  1 -1  1 -1  1  1 -1  1 -1 -1 -1  1 -1 -1 -1 -1 -1
  1  1  1 -1  1  1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1  1  1 -1 -1  1 -1
  1 -1 -1 -1 -1 -1  2 -1  2  2  2  2 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  2 -1
 -1 -1  2 -1 -1  2 -1  2  2  2  2 -1  2  2  2 -1 -1 -1  2 -1 -1 -1 -1  2
 -1 -1 -1  2 -1  2]


In [18]:
from sklearn.model_selection import KFold
folds = 3
knns = [2, 6, 12]

def CV(X, y, folds = 3):
    
    inds = np.arange(len(y))
    l_inds = inds[y>-1]

    kf = KFold(n_splits = folds, shuffle = True)

    scores = [0,0,0]
    for train_inds, test_inds in kf.split(l_inds):
        y_labels = copy.deepcopy(y)
        y_labels[l_inds[test_inds]] = -1
#         y_test = l_inds[test_inds]
        for k in range(len(knns)):
            model = LabelSpreading(kernel='knn', n_neighbors = knns[k])
            model.fit(X,y_labels)
            y_pred = model.transduction_
            y_test_pred = y_pred[l_inds[test_inds]]
            y_test = y[l_inds[test_inds]]
            acc = accuracy_score(y_test, y_test_pred)
    #         print(acc)
            scores[k] += acc
    for i in range(len(scores)):
        scores[i] /= folds
    print(scores)
    return np.argmax(scores)


Uruchom metodę

In [19]:
CV(X,y)

[0.698529411764706, 0.9399509803921569, 0.8982843137254902]


C:\Users\marek\Anaconda3\lib\site-packages\sklearn\semi_supervised\label_propagation.py:293: RuntimeWarning: invalid value encountered in true_divide
  self.label_distributions_ /= normalizer


1

ZADANIE: Napisz klasę, która pozwoli robić greed search po podanych parametrach i zwracać te wartości, które dają najwyższe accuracy. 

Zobacz: 

http://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html#sphx-glr-auto-examples-model-selection-plot-grid-search-digits-py


http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV

In [24]:
from itertools import product

class GridSearchCVSemi:
    def __init__(self, model, param_grid, cv):
        self.model = model
        self.params_grid = params_grid
        self.cv = cv
        
    def fit(self, X, y):
        inds = np.arange(len(y))
        l_inds = inds[y>-1]
        kf = KFold(n_splits = self.cv, shuffle = True)
        
        params_list = list(product(*self.params_grid.values()))
        keys = list(self.params_grid.keys())
        scores = [0] * len(params_list)
        
        for train_inds, test_inds in kf.split(l_inds):
            y_labels = copy.deepcopy(y)
            y_labels[l_inds[test_inds]] = -1
            y_test = y[l_inds[test_inds]]
            
            
            
            for i,params in enumerate(params_list):
                dic = {}
                for i,k in enumerate(keys):
                    dic[k] = params[i]
                self.model.set_params(**dic)
                self.model.fit(X,y_labels)
                y_pred = self.model.transduction_
                y_test_pred = y_pred[l_inds[test_inds]]
                y_test = y[l_inds[test_inds]]
                acc = accuracy_score(y_test, y_test_pred)
#                 print(acc)
                scores[i] += acc
        ind = np.argmax(scores)
        dic = {}
        for i,k in enumerate(keys):
            dic[k] = params_list[ind][i]
        self.model.set_params(**dic)
        self.model.fit(X,y)
        
        return self.model.transduction_

ZADANIE: Uruchamiamy model

In [26]:
params_grid = {'kernel': ['rbf'], 'gamma': [0.1, 1., 10, 20]}
gs = GridSearchCVSemi(LabelSpreading(), params_grid, 3)
y_pred = gs.fit(X,y)

In [27]:
accuracy_score(iris.target, y_pred)

0.9666666666666667

ZADANIE: Użyj napisanej klasy do porównania różnych metod semi-supervised na zbiorze digits dla 50 etykeitowanych przykładów

In [18]:
#TODO